In [2]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path

WARNING  Known issue with torch==2.4.0 on Windows with CPU, recommend upgrading to torch>=2.4.1 to resolve https://github.com/ultralytics/ultralytics/issues/15049


In [3]:
# 1. 모델 로드
model = YOLO('./results_20241124_155336/waste_detection_model/weights/best.pt')  # 학습한 모델 경로 지정

# 2. 이미지 디렉토리 설정
image_dir = './test/images/'  # 이미지 디렉토리
output_dir = './runs/detect/predict/'  # 결과 저장 디렉토리
os.makedirs(output_dir, exist_ok=True)

# 3. "rf"가 포함된 파일만 필터링
image_paths = [path for path in Path(image_dir).glob("*.jpg") if "rf" in path.stem]  # 파일 이름 조건 추가
print(f"Found {len(image_paths)} images with 'rf' in the name.")

# 4. 탐지 수행
results = []  # 전체 결과 저장 리스트
for image_path in image_paths:
    print(f"Processing: {image_path}")
    result = model.predict(source=str(image_path), save=True, device='cpu')  # 예측 수행
    results.append(result[0])  # 단일 이미지 결과 저장

print(f"Processed {len(results)} images. Results saved to {output_dir}")

# 5. 결과 확인
for idx, result in enumerate(results):
    print(f"Image {idx + 1}: {result.path}")
    print(f"Detected Classes: {[result.names[int(cls_id)] for cls_id in result.boxes.cls]}")

Found 79 images with 'rf' in the name.
Processing: test\images\-7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg

image 1/1 c:\Users\JeongYeonju\Desktop\yeonjuLab\Project\DSC \ \\YOLOv8 \test\images\-7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg: 640x640 1 PP bag, 38.1ms
Speed: 2.0ms preprocess, 38.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3
Processing: test\images\202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg

image 1/1 c:\Users\JeongYeonju\Desktop\yeonjuLab\Project\DSC \ \\YOLOv8 \test\images\202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg: 640x640 1 General Waste, 2 CleanNets, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3
Processing: test\images\202411061319-36-307974-127-355197_jpg.rf.40e66b47ea454f90ee055c740410cc1c.jpg

image 1/1 c:\Users\JeongYeonju\Desktop\yeonj

In [4]:
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items', 'Audio': 'Large Waste Items', 'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items', 'Main unit': 'Large Waste Items', 'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items', 'sewing machine': 'Large Waste Items', 'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items', 'Vacuum cleaner': 'Large Waste Items', 'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items', 'Sink': 'Large Waste Items', 'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items', 'Bed': 'Large Waste Items', 'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items', 'Cabinet': 'Large Waste Items', 'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items', 'Desk': 'Large Waste Items', 'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items', 'Display stand': 'Large Waste Items', 'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items', 'Small cabinet': 'Large Waste Items', 'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items', 'TV stand': 'Large Waste Items', 'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items', 'Air conditioner': 'Large Waste Items', 'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items', 'Electric rice cooker': 'Large Waste Items', 'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items', 'Heater': 'Large Waste Items', 'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items', 'refrigerator': 'Large Waste Items', 'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items', 'Washing machine': 'Large Waste Items', 'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items', 'Bedding items': 'Large Waste Items', 'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items', 'Clothes drying rack': 'Large Waste Items', 'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items', 'Earthenware jar': 'Large Waste Items', 'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items', 'lumber': 'Large Waste Items', 'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items', 'Piano': 'Large Waste Items', 'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items', 'Stroller': 'Large Waste Items', 'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items', 'audio cabinet': 'Large Waste Items', 'suitcase': 'Large Waste Items',
    
    'PP bag': 'PP bag',
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    'CleanNet': 'CleanNet'
}

In [10]:

def convert_labels_to_major_category(json_path, category_mapping):
    """소분류를 대분류로 변환."""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    converted_data = {}
    for image_name, annotations in data.items():
        converted_annotations = []
        for ann in annotations:
            class_name = ann["class_name"]
            if class_name in category_mapping:
                major_category = category_mapping[class_name]
                converted_annotations.append({
                    "class_name": major_category,
                    "bbox": ann["bbox"]
                })
        converted_data[image_name] = converted_annotations

    return converted_data

# 라벨링 데이터 변환 실행
json_path = './test/test_annotations_yolo.json'  # 라벨링 JSON 경로
converted_labels = convert_labels_to_major_category(json_path, category_mapping)

# 변환된 데이터 저장 (선택)
with open('./test/converted_annotations.json', 'w', encoding='utf-8') as f:
    json.dump(converted_labels, f, indent=4, ensure_ascii=False)


In [11]:
target_class = 'Large Waste Items'

In [12]:
def classify_images_by_major_category(converted_labels, target_class):
    """대분류 기준으로 이미지를 이진 분류."""
    binary_classification = {}

    for image_name, annotations in converted_labels.items():
        # 이미지의 모든 객체가 target_class인지 확인
        if any(ann["class_name"] == target_class for ann in annotations):
            binary_classification[image_name] = "Positive"
        else:
            binary_classification[image_name] = "Negative"
    
    return binary_classification

# 이진 분류 실행
binary_classification = classify_images_by_major_category(converted_labels, target_class)

# 결과 출력
for image_name, classification in binary_classification.items():
    print(f"Image: {image_name}, Classification: {classification}")


Image: 202411061348-36-313711-127-349988_jpg.rf.3581e8d79a3a050686554a20476cdb34.jpg, Classification: Negative
Image: 202411061346-36-311745-127-353092_jpg.rf.3e408eccde358b63c05fd5a29f956aab.jpg, Classification: Negative
Image: 202411111240-36-305737-127-350237_jpg.rf.07a211fd644530b35bcda797c76089f0.jpg, Classification: Positive
Image: 202411111231-36-307637-127-351064_jpg.rf.089c4d450857dd4dca741be2cf4f1cbb.jpg, Classification: Positive
Image: 202411061319-36-307974-127-355197_jpg.rf.40e66b47ea454f90ee055c740410cc1c.jpg, Classification: Negative
Image: 202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg, Classification: Positive
Image: 202411111223-36-307550-127-354681_jpg.rf.23480e44a8092c9c3311ca80c0ed1d96.jpg, Classification: Negative
Image: 202411111216-36-309841-127-353640_jpg.rf.0cfe34d4e561604f27e4d6d51709d093.jpg, Classification: Negative
Image: 202411111148-36-330942-127-338586_jpg.rf.09088434bf25f50d833296c7059228a4.jpg, Classification: Negative
I

In [13]:
def classify_results(results, target_class):
    """객체 탐지 결과를 이진 분류로 변환."""
    binary_classification = {}

    for result in results:
        image_path = result.path  # 이미지 경로
        image_name = image_path.split('\\')[-1]  # 이미지 이름만 추출
        detected_classes = [result.names[int(cls_id)] for cls_id in result.boxes.cls]

        # 이진 분류 조건: target_class 존재 여부
        if target_class in detected_classes:
            binary_classification[image_name] = "Positive"
        else:
            binary_classification[image_name] = "Negative"
    
    return binary_classification


In [14]:
def compare_results_and_labels(binary_results, binary_labels):
    """탐지 결과와 라벨링 데이터의 이진 분류 결과 비교."""
    comparison = {"match": 0, "mismatch": 0}

    for image_name, label in binary_labels.items():
        prediction = binary_results.get(image_name, "Negative")  # 탐지 결과 없으면 Negative로 기본 설정
        if prediction == label:
            comparison["match"] += 1
        else:
            comparison["mismatch"] += 1

    return comparison

# 탐지 결과와 라벨링 데이터 비교
binary_results = classify_results(results)  # 탐지 결과 이진 분류
comparison = compare_results_and_labels(binary_results, binary_classification, target_class)

print(f"Matching Results: {comparison['match']}")
print(f"Mismatched Results: {comparison['mismatch']}")


TypeError: classify_results() missing 1 required positional argument: 'target_class'

In [19]:
def evaluate_binary_classification(predicted, ground_truth):
    """
    이진 분류 결과 평가.
    Args:
        predicted: 탐지 결과의 이진 분류 (딕셔너리, {image_name: "Positive"/"Negative"}).
        ground_truth: 라벨링 데이터의 이진 분류 (딕셔너리, {image_name: "Positive"/"Negative"}).
    Returns:
        성능 지표 딕셔너리 (정확도, 정밀도, 재현율, F1-Score).
    """
    true_positive = 0
    false_positive = 0
    false_negative = 0
    true_negative = 0

    for image_name, gt_label in ground_truth.items():
        pred_label = predicted.get(image_name, "Negative")  # 탐지 결과 없으면 Negative로 처리

        if pred_label == "Positive" and gt_label == "Positive":
            true_positive += 1
        elif pred_label == "Positive" and gt_label == "Negative":
            false_positive += 1
        elif pred_label == "Negative" and gt_label == "Positive":
            false_negative += 1
        elif pred_label == "Negative" and gt_label == "Negative":
            true_negative += 1

    # 정확도 (Accuracy)
    print(true_positive, false_positive, false_negative, true_negative)
    total = true_positive + false_positive + false_negative + true_negative
    accuracy = (true_positive + true_negative) / total if total > 0 else 0

    # 정밀도 (Precision)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0

    # 재현율 (Recall)
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0

    # F1-Score
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score
    }


In [20]:
def final_performance_metrics(target_class):
    binary_results = classify_results(results, target_class=target_class)  # 탐지 결과를 이진 분류
    binary_labels = classify_images_by_major_category(converted_labels, target_class=target_class)  # 라벨링 데이터 이진 분류
    performance_metrics = evaluate_binary_classification(binary_results, binary_labels)

    print("Performance Metrics:")
    for metric, value in performance_metrics.items():
        print(f"{metric}: {value:.4f}")

In [21]:
final_performance_metrics('Large Waste Items') # 대형폐기물 존재 여부

20 5 5 49
Performance Metrics:
Accuracy: 0.8734
Precision: 0.8000
Recall: 0.8000
F1-Score: 0.8000


In [22]:
final_performance_metrics('PP bag') # PP마대 존재 여부

10 1 1 67
Performance Metrics:
Accuracy: 0.9747
Precision: 0.9091
Recall: 0.9091
F1-Score: 0.9091
